In [1]:
#Import librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math 

from scipy.stats import pearsonr
import scipy.stats as stats
from plotly.subplots import make_subplots
from scipy.stats import shapiro
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore") 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import skew, shapiro
from scipy.stats import boxcox
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_auc_score, roc_curve, auc, f1_score, mean_squared_error,
    ConfusionMatrixDisplay
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import optuna
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
import lime
from lime.lime_tabular import LimeTabularExplainer
import lightgbm as lgb



In [2]:
#Import data
train_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/train_df_cleaned_new.csv', delimiter=',', encoding='utf-8',index_col=0)
test_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/test_df_cleaned_new.csv', delimiter=',', encoding='utf-8',index_col=0)



## Data Splitting

In [3]:
X_train = train_df.drop(columns=['is_fraud'])  
y_train = train_df['is_fraud']  

X_test = test_df.drop(columns=['is_fraud'])  
y_test = test_df['is_fraud']  


In [4]:
train_df

,merchant,amt,first,last,street,city,zip,lat,long,city_pop,...,age_group_senior,amt_category_medium,amt_category_high,trans_date_trans_time_year,trans_date_trans_time_month,trans_date_trans_time_day,trans_date_trans_time_weekday,trans_date_trans_time_hour,dob_weekday,dob_hour
cc_num,,,,,,,,,,,,,,,,,,,,,
0.0,0.523121,0.605444,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.666667,1.000000,0.565217,0.0,0.0
0.0,0.325145,0.538535,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.666667,1.000000,0.695652,0.0,0.0
0.0,0.148844,0.511837,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,True,False,0.0,0.0,0.700000,0.000000,0.304348,0.0,0.0
0.0,0.539017,0.554576,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.700000,0.000000,0.652174,0.0,0.0
0.0,0.552023,0.630376,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.733333,0.166667,0.521739,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.0,0.684971,0.513723,0.085294,0.485106,0.927411,0.553066,0.608915,0.600027,0.604544,0.266659,...,True,True,False,0.0,1.0,1.000000,0.500000,0.086957,0.0,0.0
1.0,0.093931,0.150035,0.085294,0.485106,0.927411,0.553066,0.608915,0.600027,0.604544,0.266659,...,True,False,False,0.0,1.0,1.000000,0.500000,0.217391,0.0,0.0
1.0,0.234104,0.567530,0.085294,0.485106,0.927411,0.553066,0.608915,0.600027,0.604544,0.266659,...,True,False,True,0.0,1.0,1.000000,0.500000,0.478261,0.0,0.0


## 🎯 Evaluation metrics

In [5]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

In [6]:
import plotly.graph_objs as go
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_recall_curve, auc, balanced_accuracy_score, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.validation import check_is_fitted
from imblearn.metrics import geometric_mean_score
import numpy as np
from plotly.subplots import make_subplots

def evaluate_model_interactive(model, X_train, X_test, y_train, y_test, threshold_plot=False):
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    try:
        check_is_fitted(model, "predict_proba")
        y_train_proba = model.predict_proba(X_train)[:, 1]
        y_test_proba = model.predict_proba(X_test)[:, 1]
    except AttributeError:
        try:
            y_train_proba = model.decision_function(X_train)
            y_test_proba = model.decision_function(X_test)
        except AttributeError:
            print("Warning: AUC calculation skipped (no probability outputs available)")
            y_train_proba = None
            y_test_proba = None

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    overall_accuracy = accuracy_score(np.concatenate([y_train, y_test]), np.concatenate([y_train_pred, y_test_pred]))

    balanced_acc_train = balanced_accuracy_score(y_train, y_train_pred)
    balanced_acc_test = balanced_accuracy_score(y_test, y_test_pred)
    balanced_acc_overall = (balanced_acc_train + balanced_acc_test) / 2

    f1 = f1_score(y_test, y_test_pred)
    g_mean = geometric_mean_score(y_test, y_test_pred)

    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
    class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
    recalls = recall_score(y_test, y_test_pred, average=None)

    wba_train = sum(class_weight_dict[i] * recall_score(y_train, y_train_pred, average=None)[i] for i in range(len(recalls))) / sum(class_weights)
    wba_test = sum(class_weight_dict[i] * recalls[i] for i in range(len(recalls))) / sum(class_weights)
    wba_overall = (wba_train + wba_test) / 2

    mcc = matthews_corrcoef(y_test, y_test_pred)

    print(f"\033[1;34mAlgorithm:\033[0m {type(model).__name__}")
    print("\n\033[1;32mClassification Report:\033[0m\n", classification_report(y_test, y_test_pred, target_names=["No Fraud", "Fraud"]))
    print(f"\033[1;36mTraining Accuracy:\033[0m {train_accuracy:.4f}")
    print(f"\033[1;36mTest Accuracy:\033[0m {test_accuracy:.4f}")
    print(f"\033[1;36mOverall Accuracy:\033[0m {overall_accuracy:.4f}")

    print(f"\033[1;35mG-Mean:\033[0m {g_mean:.4f}")
    print(f"\033[1;35mMatthews Correlation Coefficient (MCC):\033[0m {mcc:.4f}")

    if y_test_proba is not None:
        precision_train, recall_train, _ = precision_recall_curve(y_train, y_train_proba)
        pr_auc_train = auc(recall_train, precision_train)

        precision_test, recall_test, _ = precision_recall_curve(y_test, y_test_proba)
        pr_auc_test = auc(recall_test, precision_test)

        recall_combined = np.concatenate([recall_train, recall_test])
        precision_combined = np.concatenate([precision_train, precision_test])

        sorted_indices = np.argsort(recall_combined)
        recall_combined = recall_combined[sorted_indices]
        precision_combined = precision_combined[sorted_indices]

        print(f"\033[1;31mTraining Precision-Recall AUC:\033[0m {pr_auc_train:.4f}")
        print(f"\033[1;31mTest Precision-Recall AUC:\033[0m {pr_auc_test:.4f}")
        print(f"\033[1;31mOverall Precision-Recall AUC:\033[0m {auc(recall_combined, precision_combined):.4f}")

        trace1 = go.Scatter(
            x=recall_test, y=precision_test,
            name="Test Precision-Recall Curve",
            line=dict(color='blue', width=3),
            hovertemplate='<b>Recall:</b> %{x}<br><b>Precision:</b> %{y}'
        )

        trace2 = go.Scatter(
            x=recall_train, y=precision_train,
            name="Train Precision-Recall Curve",
            line=dict(color='green', width=3),
            hovertemplate='<b>Recall:</b> %{x}<br><b>Precision:</b> %{y}'
        )

        trace3 = go.Scatter(
            x=recall_combined, y=precision_combined,
            name="Overall Precision-Recall Curve",
            line=dict(color='purple', width=3, dash='dot'),
            hovertemplate='<b>Recall:</b> %{x}<br><b>Precision:</b> %{y}'
        )

        trace4 = go.Scatter(
            x=[0, 1], y=[1, 0],
            name="Random Chance",
            line=dict(color='red', dash='dash', width=2)
        )

    conf_matrix = confusion_matrix(y_test, y_test_pred)

    heatmap = go.Heatmap(
        z=conf_matrix,
        x=["No fraud", "fraud"],
        y=["No fraud", "fraud"],
        colorscale=[
            [0.0, "#FFF7EC"],
            [0.33, "#FDBB84"],
            [0.66, "#FC8D59"],
            [1.0, "#D7301F"]
        ],
        zmin=0,
        zmax=conf_matrix.max(),
        colorbar=dict(title="Count"),
        hovertemplate="True Class: %{y}<br>Predicted: %{x}<br>Count: %{z}<extra></extra>",
        showscale=True
    )

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'Confusion Matrix',
            'Precision-Recall Curve'
        )
    )

    fig.add_trace(heatmap, row=1, col=1)
    fig.add_trace(trace1, row=1, col=2)
    fig.add_trace(trace2, row=1, col=2)
    fig.add_trace(trace3, row=1, col=2)
    fig.add_trace(trace4, row=1, col=2)

    fig.update_layout(
        showlegend=False,
        title="Model Performance",
        autosize=True,
        height=600, width=1000,
        plot_bgcolor='rgba(240,240,240,0.95)',
        paper_bgcolor='rgba(240,240,240,0.95)',
        font=dict(family="Arial", size=12),
    )

    fig["layout"]["xaxis1"].update(title="Predicted Labels")
    fig["layout"]["yaxis1"].update(title="Actual Labels")
    fig["layout"]["xaxis2"].update(title="Recall")
    fig["layout"]["yaxis2"].update(title="Precision")

    fig.show()


<div style="text-align: center; background-color: #e0f7fa; padding: 20px; border-radius: 10px; font-size: 24px; font-weight: bold; color: #004d40;">
    🌿 Tree Ensembles
</div>


## LightGBM

In [7]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import average_precision_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 500, 1000, step=50)
    max_depth = trial.suggest_int('max_depth', 7, 15)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    subsample = trial.suggest_float('subsample', 0.7, 0.9)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 0.9)
    min_split_gain = trial.suggest_float('min_split_gain', 0.0, 0.5)  
    reg_alpha = trial.suggest_float('reg_alpha', 10, 30)  
    reg_lambda = trial.suggest_float('reg_lambda', 10, 30)  
    num_leaves = trial.suggest_int('num_leaves', 50, 300, step=10)  
    min_child_samples = trial.suggest_int('min_child_samples', 200, 500, step=10)  
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 500, step=10) 

    model = lgb.LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        min_split_gain=min_split_gain,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        num_leaves=num_leaves,
        min_child_samples=min_child_samples,
        min_data_in_leaf=min_data_in_leaf,  
        objective='binary',
        metric='auc',
        class_weight=class_weights,
        random_state=42,
        verbose=-1,
        n_jobs=-1
    )

    stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    auc_scores = []
    
    for train_index, val_index in stratified_kfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        model.fit(X_train_fold, y_train_fold)
        y_pred_fold = model.predict_proba(X_val_fold)[:, 1]  
        auc_scores.append(roc_auc_score(y_val_fold, y_pred_fold))
                          
    return np.mean(auc_scores)


In [8]:
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=15)  

best_trial = study.best_trial
print(f"Best trial: {best_trial.params}")


LightGBM = lgb.LGBMClassifier(
    n_estimators=best_trial.params['n_estimators'],
    max_depth=best_trial.params['max_depth'],
    learning_rate=best_trial.params['learning_rate'],
    subsample=best_trial.params['subsample'],
    colsample_bytree=best_trial.params['colsample_bytree'],
    min_split_gain=best_trial.params['min_split_gain'],
    reg_alpha=best_trial.params['reg_alpha'],
    reg_lambda=best_trial.params['reg_lambda'],
    num_leaves=best_trial.params['num_leaves'],
    min_child_samples=best_trial.params['min_child_samples'],
    min_data_in_leaf=best_trial.params['min_data_in_leaf'],
    objective='binary',
    metric='auc',
    class_weight=class_weights,
    random_state=42,
    verbose=-1,
    n_jobs=-1
)

LightGBM.fit(X_train, y_train)

[I 2025-10-07 18:08:33,631] A new study created in memory with name: no-name-e293bbf7-5368-4b7b-a061-d774153af286
[I 2025-10-07 18:12:49,411] Trial 0 finished with value: 0.9995893359810619 and parameters: {'n_estimators': 1000, 'max_depth': 9, 'learning_rate': 0.07294860109811416, 'subsample': 0.7851473498822601, 'colsample_bytree': 0.771212246541126, 'min_split_gain': 0.3105199172327782, 'reg_alpha': 16.471092285961134, 'reg_lambda': 21.100933955626317, 'num_leaves': 220, 'min_child_samples': 320, 'min_data_in_leaf': 480}. Best is trial 0 with value: 0.9995893359810619.
[I 2025-10-07 18:16:11,344] Trial 1 finished with value: 0.9995943849425422 and parameters: {'n_estimators': 750, 'max_depth': 8, 'learning_rate': 0.09125476095326206, 'subsample': 0.8177359301156502, 'colsample_bytree': 0.6661436608279097, 'min_split_gain': 0.29366839895406777, 'reg_alpha': 19.964874127267848, 'reg_lambda': 23.139627854330307, 'num_leaves': 190, 'min_child_samples': 260, 'min_data_in_leaf': 210}. Bes

KeyboardInterrupt: 

In [ ]:
evaluate_model_interactive(
    LightGBM, 
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    threshold_plot=False
)

In [11]:
#save model
import pickle
import os

save_path = r"C:\Users\Mega-Pc\Freelance\data\lightgbm_model.pkl"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, "wb") as f:
    pickle.dump(LightGBM, f)

print(f"Model saved at: {save_path}")


Model saved at: C:\Users\Mega-Pc\Freelance\data\lightgbm_model.pkl
